In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("./tools/")
import commontools
import ucTools
import funclib
import time
import joblib

import train
import test

%load_ext autoreload
%autoreload 2

In [30]:
uctools = ucTools.ucTools('172.16.25.20')
cnx_ecdb = uctools.db_conn()

# 1. 加载序列序列文件
file_1515 = './tasks/task3/data/ecoli1515/1515testset_with_unirep.tsv'
data_1515 = pd.read_csv(file_1515, sep='\t', index_col=0)

colname = [
    ['id',
     'ec_number',
     'seq',
     'isenzyme',
     'isMultiFunctional',
     'blattner_id',
     'blattner_ec_number'] + ['f' + str(i) for i in range(1, 1901)]]

data_1515.columns = np.array(colname)[0]

file_ref = './data/preprocess/enzyme_train.fasta'
blast_res_file = './results/1515/balastres.tsv'

# 2. 获取序列比对结果
blast_res = test.getblast(querydata=data_1515, refdata=file_ref, blastres_file=blast_res_file)

# 3.获取酶-非酶预测结果
isEnzyme_model = './model/isenzyme.model'
isEnzyme_pred, isEnzyme_pred_prob = test.get_isEnzymeRes(querydata=data_1515.iloc[:, 8:1909], model_file=isEnzyme_model)

# 4.获取Slice预测结果
slice_pred = test.get_slice_res(data_1515.iloc[:, 7:1909], './model', './results/1515/slice_results.txt')

# 5. 获取最终结果
file_label_ec = './data/dict_label_ec.npy'
final_pred = test.get_final_results(ori_data=data_1515,
                               blast_data=blast_res,
                               isenzyme_data=isEnzyme_pred,
                               slice_data=slice_pred,
                               cnx=cnx_ecdb,
                               file_label_ec=file_label_ec
                               )

./slice_predict ./data/preprocess/slice_x_1515.txt ./model ./results/1515/slice_results.txt -o 32 -b 0 -t 32 -q 0


/home/shizhenkun/codebase/BioUniprot/test.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ori_data['isenzyme_slice'] = isenzyme_data


In [31]:
final_pred

,id,ec_number,isenzyme_slice,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,0,1,...,10,11,12,13,14,15,16,17,18,19
0,P42641,3.6.5.-,1,100.0,True,False,1.0,3.6.5.-,3.6.5.2,1.18.6.1,...,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27
1,P0AE22,3.1.3.2,1,100.0,True,False,1.0,3.1.3.2,3.1.3.2,2.5.1.31,...,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39
2,P21499,3.1.13.1,1,100.0,True,False,1.0,3.1.13.1,5.2.1.8,3.1.13.1,...,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136
3,P39286,3.6.1.-,1,100.0,True,False,1.0,3.6.1.-,2.9.1.3,2.7.4.33,...,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23
4,P56100,7.1.1.7,0,100.0,True,False,1.0,7.1.1.7,7.1.1.2,7.1.1.7,...,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7131,P36837,NaN,0,100.0,False,False,1.0,-,7.2.2.10,2.7.1.202,...,2.4.1.17,3.4.16.5,2.7.7.74,2.7.8.34,2.1.1.86,7.1.1.9,2.7.7.41,2.7.1.66,4.1.1.111,6.3.3.3
7132,P77304,NaN,0,100.0,False,False,1.0,-,7.2.2.10,7.2.2.8,...,1.8.7.3,3.1.3.4,1.21.99.4,2.1.1.86,1.8.98.6,7.2.2.13,2.7.13.3,1.2.5.3,7.2.2.4,4.1.1.111
7133,P27248,2.1.2.10,1,100.0,True,False,1.0,2.1.2.10,2.1.2.10,2.7.2.11,...,2.8.3.9,2.5.1.94,4.1.2.42,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57
7134,P0AC88,4.2.1.47,1,100.0,True,False,1.0,4.2.1.47,4.2.1.47,2.4.1.62,...,3.7.1.12,2.7.1.157,4.1.1.105,4.1.99.5,3.6.1.64,5.1.3.15,1.14.13.25,5.1.3.26,2.1.1.179,1.1.1.308


In [36]:
final_pred[(final_pred.isenzyme_slice != final_pred.isenzyme_blast) |((final_pred.isenzyme_slice ==0) & (final_pred.isenzyme_blast.isnull()))]

,id,ec_number,isenzyme_slice,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,0,1,...,10,11,12,13,14,15,16,17,18,19
4,P56100,7.1.1.7,0,100.0,True,False,1.0,7.1.1.7,7.1.1.2,7.1.1.7,...,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5
9,Q46865,3.1.-.-,0,100.0,True,False,1.0,3.1.-.-,2.7.8.7,3.5.1.135,...,3.6.1.56,5.3.3.4,3.5.1.5,4.2.1.104,3.5.4.35,2.5.1.107,4.1.3.40,1.14.11.33,1.14.11.64,4.2.1.94
10,P0AE70,3.1.27.-,0,100.0,True,False,1.0,3.1.27.-,2.6.1.42,3.6.1.29,...,3.2.2.17,6.3.2.41,3.4.19.15,5.4.99.43,2.4.1.16,2.4.2.27,5.1.3.15,4.3.2.9,2.7.1.26,3.6.1.67
14,P37690,NaN,1,100.0,False,False,1.0,-,3.4.24.75,3.4.14.13,...,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45
28,P77737,NaN,1,100.0,False,False,1.0,-,7.4.2.9,7.4.2.10,...,3.6.1.71,1.3.1.2,7.2.2.2,3.1.12.1,4.1.99.23,1.8.98.6,1.12.98.1,1.2.7.7,1.3.5.1,1.4.99.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7097,Q2EER5,NaN,1,100.0,False,False,1.0,-,1.1.1.37,2.1.1.113,...,2.4.2.9,5.4.99.18,1.18.1.2,2.7.4.27,2.7.11.32,1.14.19.9,2.7.1.205,2.1.1.211,5.3.1.34,1.20.4.1
7106,P64612,NaN,1,100.0,False,False,1.0,-,7.1.2.2,3.6.4.12,...,6.6.1.2,7.4.2.4,2.7.4.13,2.7.1.163,2.1.1.160,3.1.21.3,1.4.1.17,1.14.14.8,3.4.22.44,2.3.1.38
7111,P0DSG3,NaN,0,NaN,NaN,NaN,NaN,NaN,7.1.1.2,4.1.3.40,...,1.14.11.33,5.4.99.20,5.1.99.7,1.3.7.6,4.4.1.17,3.1.12.1,3.1.21.4,3.6.1.55,3.5.1.135,3.3.2.11
7113,P0DSG9,NaN,0,NaN,NaN,NaN,NaN,NaN,1.14.13.81,3.4.17.6,...,2.3.2.34,2.5.1.103,6.3.1.20,3.2.2.21,3.6.1.67,4.4.1.20,7.2.4.2,6.1.1.20,1.3.7.3,4.3.2.9


In [38]:
final_pred[(final_pred.isenzyme_slice==1) &(final_pred.isenzyme_blast==False)]

,id,ec_number,isenzyme_slice,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,0,1,...,10,11,12,13,14,15,16,17,18,19
14,P37690,NaN,1,100.0,False,False,1.0,-,3.4.24.75,3.4.14.13,...,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45
28,P77737,NaN,1,100.0,False,False,1.0,-,7.4.2.9,7.4.2.10,...,3.6.1.71,1.3.1.2,7.2.2.2,3.1.12.1,4.1.99.23,1.8.98.6,1.12.98.1,1.2.7.7,1.3.5.1,1.4.99.5
118,P52130,NaN,1,100.0,False,False,1.0,-,3.5.4.16,2.7.7.6,...,1.16.3.1,2.3.1.133,1.12.1.2,3.1.4.37,2.3.1.33,5.4.99.43,3.6.1.58,2.7.7.96,5.4.99.29,3.6.1.64
124,P76555,2.7.2.1,1,100.0,False,False,1.0,-,4.1.1.65,3.5.3.26,...,1.13.11.53,4.4.1.3,2.7.1.23,2.3.1.156,1.13.11.24,3.4.11.19,3.7.1.2,4.2.1.119,2.4.1.230,4.2.1.141
129,P10346,NaN,1,100.0,False,False,1.0,-,7.3.2.1,7.6.2.15,...,7.2.2.2,7.2.2.11,7.6.2.7,7.2.2.16,7.5.2.8,7.5.2.11,7.5.2.9,7.6.2.8,7.6.2.13,2.7.8.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7089,P36928,NaN,1,100.0,False,False,1.0,-,2.7.1.33,2.7.2.7,...,2.7.1.2,2.7.1.188,2.7.1.17,2.7.1.63,2.7.1.53,3.2.1.183,2.7.2.1,2.7.1.51,3.5.1.124,2.7.1.215
7090,P52123,NaN,1,100.0,False,False,1.0,-,4.3.2.1,2.5.1.1,...,3.1.7.10,2.7.7.42,2.7.1.158,4.2.3.54,4.1.99.13,3.4.22.68,4.2.3.75,2.7.7.89,6.4.1.2,4.2.3.76
7097,Q2EER5,NaN,1,100.0,False,False,1.0,-,1.1.1.37,2.1.1.113,...,2.4.2.9,5.4.99.18,1.18.1.2,2.7.4.27,2.7.11.32,1.14.19.9,2.7.1.205,2.1.1.211,5.3.1.34,1.20.4.1
7106,P64612,NaN,1,100.0,False,False,1.0,-,7.1.2.2,3.6.4.12,...,6.6.1.2,7.4.2.4,2.7.4.13,2.7.1.163,2.1.1.160,3.1.21.3,1.4.1.17,1.14.14.8,3.4.22.44,2.3.1.38


In [40]:
cc=final_pred[(final_pred.isenzyme_slice==1) &(final_pred.isenzyme_blast==False)]

In [41]:
cc.to_excel('./results/1515/mayEnzyme.xlsx', index=None)

In [43]:
final_pred[(final_pred.isenzyme_slice==0) & (final_pred.isenzyme_blast==True)]

,id,ec_number,isenzyme_slice,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,0,1,...,10,11,12,13,14,15,16,17,18,19
4,P56100,7.1.1.7,0,100.0,True,False,1.0,7.1.1.7,7.1.1.2,7.1.1.7,...,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5
9,Q46865,3.1.-.-,0,100.0,True,False,1.0,3.1.-.-,2.7.8.7,3.5.1.135,...,3.6.1.56,5.3.3.4,3.5.1.5,4.2.1.104,3.5.4.35,2.5.1.107,4.1.3.40,1.14.11.33,1.14.11.64,4.2.1.94
10,P0AE70,3.1.27.-,0,100.0,True,False,1.0,3.1.27.-,2.6.1.42,3.6.1.29,...,3.2.2.17,6.3.2.41,3.4.19.15,5.4.99.43,2.4.1.16,2.4.2.27,5.1.3.15,4.3.2.9,2.7.1.26,3.6.1.67
38,P76053,3.1.-.-,0,100.0,True,False,1.0,3.1.-.-,3.1.26.4,6.1.1.21,...,2.3.1.198,6.5.1.6,3.2.2.1,3.2.2.26,5.3.1.34,3.1.26.8,2.1.1.133,2.7.1.160,1.14.19.9,3.3.2.1
72,P09169,3.4.23.49,0,100.0,True,False,1.0,3.4.23.49,3.4.23.49,3.4.23.48,...,3.4.22.15,3.1.2.2,3.4.22.43,3.2.1.81,3.2.1.96,3.4.22.38,3.4.21.49,3.2.1.94,4.2.2.8,4.2.2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6753,P31992,5.3.2.-,0,100.0,True,False,1.0,5.3.2.-,2.7.7.6,4.2.1.96,...,3.6.1.7,1.14.13.69,1.14.13.149,2.7.7.46,3.6.1.65,5.4.99.43,4.2.99.23,1.14.11.11,2.3.1.204,1.14.13.7
6757,P0A9I8,1.7.1.15,0,100.0,True,False,1.0,1.7.1.15,1.7.1.15,3.6.1.7,...,2.4.1.231,3.1.13.5,5.1.3.13,4.2.1.106,1.13.11.25,4.2.3.12,4.3.2.3,2.4.2.3,3.6.1.55,2.4.2.2
6760,P07001,7.1.1.1,0,100.0,True,False,1.0,7.1.1.1,7.1.1.1,2.3.1.275,...,2.7.1.202,7.2.1.3,2.7.7.67,2.7.8.41,7.1.1.9,7.2.4.1,2.7.1.201,2.5.1.150,2.7.1.195,2.7.1.174
6780,P75995,3.1.4.52,0,100.0,True,False,1.0,3.1.4.52,3.1.4.52,3.11.1.1,...,2.3.1.268,2.7.4.13,6.3.2.1,3.1.3.104,2.7.1.212,2.7.7.65,2.4.99.13,3.1.2.31,3.6.1.1,3.1.1.82


In [1]:
from jax_unirep import get_reps



In [8]:
sequences = ["YJKAL", "YJKAL"]
h_avg, h_final, c_final= get_reps(sequences)

In [9]:
h_avg

array([[ 0.2402493 ,  0.08654933, -0.03837042, ...,  0.25523344,
         0.37647662,  0.10894282],
       [ 0.2402493 ,  0.08654933, -0.03837042, ...,  0.25523344,
         0.37647662,  0.10894282]], dtype=float32)